# Controls Example

If you want to run this notebook on your Raspberry Pi, but don't have Jupyter Notebook, you can install it with `pip`.

`pip install jupyter`

Then to run it, simply enter `jupyter-notebook` into the Raspberry Pi terminal
and then navigate the `picamera2/examples/notebooks` folder to find this notebook.


## Setup

In [1]:
from picamera2.picamera2 import *
picam2 = Picamera2()

[0:18:18.889055277] [1775]  INFO Camera camera_manager.cpp:293 libcamera v0.0.0+3548-a11d63f9
[0:18:18.926675706] [1789]  WARN CameraSensorProperties camera_sensor_properties.cpp:163 No static properties available for 'imx477'
[0:18:18.926714539] [1789]  WARN CameraSensorProperties camera_sensor_properties.cpp:165 Please consider updating the camera sensor properties database
[0:18:18.926733834] [1789] ERROR CameraSensor camera_sensor.cpp:591 'imx477 10-001a': Camera sensor does not support test pattern modes.
[0:18:18.944477746] [1789]  INFO RPI raspberrypi.cpp:1352 Registered camera /base/soc/i2c0mux/i2c@1/imx477@1a to Unicam device /dev/media2 and ISP device /dev/media0


## Method #1: Using the set_controls function.

In [2]:
my_controls = {'NoiseReductionMode': 2,
               'Contrast': 1,
               'Sharpness': 1}

picam2.set_controls(my_controls)

The `Controls` class is automatically brought into the `Picamera2` class, so for most applications you probably won't need to
call it individually. It is renamed to `controls` when using it through the `Picamera2`.
To check the controls, you can use `controls.config`. The nice thing about this is that the function automatically compiles the controls you adjusted into a dictionary and checks their validity (within reason).

In [3]:
picam2.controls.config

{'Contrast': 1, 'Sharpness': 1, 'NoiseReductionMode': 2}

## Method #2: Using class attributes.
Controls are imported from the `Controls` class, which treats each control as an attribute or property of the class.

In [4]:
picam2.controls.NoiseReductionMode = 1
picam2.controls.Contrast = 0
picam2.controls.Sharpness = 0

picam2.controls.config

{'Contrast': 0, 'Sharpness': 0, 'NoiseReductionMode': 1}

## Control Input Options

In most cases, `the Controls` class will notify you if the control you input is the incorrect value or type when
the camera reads in the controls. There are several ways you can input values for each control, but it depends on the control.

To determine what the input types are for each control, you can use the annotation attribute of the `Controls` class. If you see something like (int,str) that means you can supply the control input as an integer OR string.

As of writing this notebook, control names and enums must match the syntax of what libcamera uses.
For example, if you input "off", an error will be thrown by libcamera. The correct input would be "Off".
Another example would `picam2.controls.noisereductionmode` which would not register as a control option. The correct input would be `picam2.controls.NoiseReductionMode`.

In [5]:
picam2.controls.anno

{'AeEnable': (int, bool),
 'AeMeteringMode': (int, str),
 'AeConstraintMode': (int, str),
 'AeExposureMode': (int, str),
 'ExposureValue': (float, int),
 'ExposureTime': (float, int),
 'AnalogueGain': (float, int),
 'Brightness': (float, int),
 'Contrast': (float, int),
 'AwbEnable': (int, bool),
 'AwbMode': (int, str),
 'ColourGains': (tuple, list),
 'Saturation': (float, int),
 'Sharpness': (float, int),
 'ColourCorrectionMatrix': (tuple, list),
 'ScalerCrop': (tuple, list),
 'DigitalGain': (float, int),
 'FrameDurationLimits': (tuple, list),
 'AePrecaptureTrigger': int,
 'AfTrigger': int,
 'NoiseReductionMode': (int, str),
 'ColorCorrectionAberrationMode': (int, str),
 'SceneFlicker': (int, str),
 'PipelineDepth': int,
 'MaxLatency': int,
 'TestPatternMode': (int, str)}

For controls that configure camera modes, you can supply an integer, an enum, or string.

In [6]:
#These all turn off NoiseReductionMode.
picam2.controls.NoiseReductionMode = 0
picam2.controls.NoiseReductionMode = libcamera.NoiseReductionMode.Off
picam2.controls.NoiseReductionMode = 'Off'

For controls that enable or disable a function, you can supply an integer or bool.

In [7]:
#These lines disable auto exposure mode.

picam2.controls.AeEnable = 0
picam2.controls.AeEnable = False

For controls that require an array as input, you can supply it as a tuple or a list.

In [8]:
#These both set the FrameDurationLimits to 33333 or 30 frames per second.

picam2.controls.FrameDurationLimits = (33333,33333)
picam2.controls.FrameDurationLimits = [33333,33333]

## Configuring the Controls
At this stage, we actually haven't sent the controls to the camera. The neat thing is that we don't
need to do this. This is done automatically when you tell the camera to start.

In [9]:
picam2.controls.clear #You may not need this. It only clears the controls BEFORE the camera is started.
picam2.controls.NoiseReductionMode = 1

picam2.configure(picam2.preview_configuration())
picam2.start_preview(Preview.QTGL)
picam2.start()


[0:18:19.167344748] [1775]  INFO Camera camera.cpp:1029 configuring streams: (0) 640x480-XBGR8888
[0:18:19.167802369] [1789]  INFO RPI raspberrypi.cpp:760 Sensor: /base/soc/i2c0mux/i2c@1/imx477@1a - Selected sensor format: 2028x1520-SBGGR12_1X12 - Selected unicam format: 2028x1520-pBCC
[0:18:20.939766348] [1798]  INFO IPARPI raspberrypi.cpp:635 Request ctrl: NoiseReductionMode = 2
qt.qpa.xcb: QXcbConnection: XCB error: 148 (Unknown), sequence: 192, resource id: 0, major code: 140 (Unknown), minor code: 20


## Control Manipulation

All of the controls are technically class variables. That means that all instances of the class share the same values.
So controls can be updated by either method after the camera has already started, without having to start the camera again.

In [10]:
for i in range(0,20):
    j = i/20 + 0.05
    picam2.controls.Brightness = j
    time.sleep(1)

[0:18:22.053895820] [1798]  INFO IPARPI raspberrypi.cpp:635 Request ctrl: NoiseReductionMode = 2
[0:18:22.053950115] [1798]  INFO IPARPI raspberrypi.cpp:635 Request ctrl: Brightness = 0.050000
[0:18:22.723722336] [1798]  INFO IPARPI raspberrypi.cpp:635 Request ctrl: Brightness = 0.100000
[0:18:24.012997749] [1798]  INFO IPARPI raspberrypi.cpp:635 Request ctrl: Brightness = 0.150000
[0:18:25.005245387] [1798]  INFO IPARPI raspberrypi.cpp:635 Request ctrl: Brightness = 0.200000
[0:18:26.004152116] [1798]  INFO IPARPI raspberrypi.cpp:635 Request ctrl: Brightness = 0.250000
[0:18:27.003592425] [1798]  INFO IPARPI raspberrypi.cpp:635 Request ctrl: Brightness = 0.300000
[0:18:28.005935804] [1798]  INFO IPARPI raspberrypi.cpp:635 Request ctrl: Brightness = 0.350000
[0:18:29.005101668] [1798]  INFO IPARPI raspberrypi.cpp:635 Request ctrl: Brightness = 0.400000
[0:18:30.005799037] [1798]  INFO IPARPI raspberrypi.cpp:635 Request ctrl: Brightness = 0.450000
[0:18:31.006512346] [1798]  INFO IPARPI

True

Then to check the current controls we can use the `current` function.

In [ ]:
picam2.controls.current

## Current Settings

After the settings that you assign are passed to the camera, the Controls()
class is effectively reset so that there is no risk of repetition. However,
the Controls() class does keep track of the current settings of the camera.
The current settings are always the settings that you last sent to the camera and that were accepted.

If we look at the current settings now, we should see that the NoiseReductionMode is set to 2
and the Brightness is 1.

## Resetting Controls (in development)

If you wanted to reset the controls to what they were when you first turned on the camera,
you can issue the `reset` command. But first, we need to find out what the default controls were. If you compare it to the `ranges` function in the following cell, you may notice that some values are 0 when the 0 is technically outside the limits. This just means that the camera is handling these internally or in an automatic mode.

In [12]:
picam2.controls.defaults #This is a read-only property and will reflect whatever comes out of Picamera2().camera.controls.

{'FrameDurationLimits': [0, 0],
 'ColourCorrectionMatrix': [0, 0, 0, 0, 0, 0, 0, 0, 0],
 'NoiseReductionMode': 0,
 'Contrast': 1.0,
 'AwbMode': 0,
 'ScalerCrop': (0, 0, 0, 0),
 'AwbEnable': 0,
 'ExposureValue': 0.0,
 'ColourGains': [0, 0],
 'AeExposureMode': 0,
 'Sharpness': 1.0,
 'Brightness': 0.0,
 'AeConstraintMode': 0,
 'AeMeteringMode': 0,
 'AnalogueGain': 0,
 'Saturation': 1.0,
 'ExposureTime': 0,
 'AeEnable': 0}

In [13]:
picam2.controls.reset #Read only. Accepts no inputs.
picam2.controls.config

{'AeEnable': 0,
 'AeMeteringMode': 0,
 'AeConstraintMode': 0,
 'AeExposureMode': 0,
 'ExposureValue': 0.0,
 'ExposureTime': 0,
 'AnalogueGain': 0,
 'Brightness': 0.0,
 'Contrast': 1.0,
 'AwbEnable': 0,
 'AwbMode': 0,
 'ColourGains': [0, 0],
 'Saturation': 1.0,
 'Sharpness': 1.0,
 'ColourCorrectionMatrix': [0, 0, 0, 0, 0, 0, 0, 0, 0],
 'ScalerCrop': (0, 0, 0, 0),
 'FrameDurationLimits': [0, 0],
 'NoiseReductionMode': 0}

We can see that the changes in the output reflect the values in the defaults.

## Control Ranges (in development)
To get an idea of the valid ranges for each control, you can use the ranges function.

In [14]:
picam2.controls.ranges #Read only.

{'FrameDurationLimits': {'min': 1000, 'max': 1000000000},
 'ColourCorrectionMatrix': {'min': -16.0, 'max': 16.0},
 'NoiseReductionMode': {'min': 0, 'max': 4},
 'Contrast': {'min': 0.0, 'max': 32.0},
 'AwbMode': {'min': 0, 'max': 7},
 'ScalerCrop': {'min': (0, 0, 0, 0), 'max': (65535, 65535, 65535, 65535)},
 'AwbEnable': {'min': False, 'max': True},
 'ExposureValue': {'min': -8.0, 'max': 8.0},
 'ColourGains': {'min': 0.0, 'max': 32.0},
 'AeExposureMode': {'min': 0, 'max': 3},
 'Sharpness': {'min': 0.0, 'max': 16.0},
 'Brightness': {'min': -1.0, 'max': 1.0},
 'AeConstraintMode': {'min': 0, 'max': 3},
 'AeMeteringMode': {'min': 0, 'max': 3},
 'AnalogueGain': {'min': 1.0, 'max': 32.0},
 'Saturation': {'min': 0.0, 'max': 32.0},
 'ExposureTime': {'min': 0, 'max': 999999},
 'AeEnable': {'min': False, 'max': True}}

## End of Example

In [11]:
picam2.stop_preview()
picam2.stop()
picam2.close()